In [1]:
#Modeling - Initial time based modeling
#Goal of this notebook is to read in a training file, select out a sport from the activities,
#Plot it and predict on what the performance at a later date will be.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date